# Notebook for recalculating magnetometer data files

in case there is error upon data entry. e.g. wrong magnetic_core_strike or sun compass orientations, we can take the specimen coordinate data from magnetometer data files and recalculate orientations in geographic and tilt-corrected coordinates.

In [1]:
import numpy as np
import pandas as pd
import pmagpy.pmag as pmag 

In [5]:
HF1_orientations = pd.read_csv('Jacobsville_corrected/HF1-.csv', header = 6)
HF1_orientations.head()

,sample_name,comment,strat_level,magnetic_core_strike,core_dip,bedding_strike,bedding_dip,correct_bedding_using_local_dec,shadow_angle,GMT_offset,...,hours,minutes,mass,runs,sun_core_strike,calculated_IGRF,IGRF_local_dec,calculated_mag_dec,core_strike,corrected_bedding_strike
0,0,mag compass orientation (IGRF corrected),80.60,343,78.0,208.5,9.0,no,NaN,-4.0,...,1.0,1.0,1.0,NaN,NaN,[355.8069954677092; 72.71224071686532; 55597.5...,-4.193005,insufficient data,338.8,NaN
1,1,sun compass orientation,80.05,302,9.0,208.5,9.0,no,9.0,-4.0,...,10.0,54.0,1.0,NaN,297.2,[355.80698842065897; 72.71215033749395; 55597....,-4.193012,-4.800000000000011,297.2,NaN
2,2,sun compass orientation,80.23,323,81.0,208.5,9.0,no,31.0,-4.0,...,11.0,0.0,1.0,NaN,320.6,[355.80698834935686; 72.71214942302974; 55597....,-4.193012,-2.3999999999999773,320.6,NaN
3,3,mag compass orientation (IGRF corrected),80.24,274,71.0,208.5,9.0,no,NaN,-4.0,...,1.0,1.0,1.0,NaN,NaN,[355.8069954677092; 72.71224071686532; 55597.5...,-4.193005,insufficient data,269.8,NaN
4,4,mag compass orientation (IGRF corrected),80.25,328,77.0,208.5,9.0,no,NaN,-4.0,...,11.0,14.0,1.0,NaN,NaN,[355.8069881829852; 72.7121472892797; 55597.40...,-4.193012,insufficient data,323.8,NaN


In [109]:
sam_file = open('Jacobsville_correction/HF1-.sam')
sam_file_lines = sam_file.readlines()[2:]
for i in range(len(sam_file_lines)):
    this_file_path = sam_file_lines[i].split('\n')[0]
    corrected_file = open('Jacobsville_corrected/'+this_file_path, 'r')
    corrected_file_lines = corrected_file.readlines()[:2]
    correct_core_strike = float(corrected_file_lines[1].split()[1])
    correct_core_dip = float(corrected_file_lines[1].split()[2])
    bedding_strike = float(corrected_file_lines[1].split()[3])
    bedding_dip = float(corrected_file_lines[1].split()[4])
    
    incorrect_file = open('Jacobsville_correction/'+this_file_path)
    incorrect_file_lines = incorrect_file.readlines()[2:]
    for j in range(len(incorrect_file_lines)):
        heading = incorrect_file_lines[j][:7]
        moment_csd = incorrect_file_lines[j][31:46]

        this_spec_dec = float(incorrect_file_lines[j][45:51])
        this_spec_inc = float(incorrect_file_lines[j][51:57])
        this_no_change_text = incorrect_file_lines[j][57:]
        correct_geo_dec = np.round(pmag.dogeo(this_spec_dec, this_spec_inc, correct_core_strike-90,  -correct_core_dip), 1)[0]
        correct_geo_inc = np.round(pmag.dogeo(this_spec_dec, this_spec_inc, correct_core_strike-90,  -correct_core_dip), 1)[1]
        correct_strat_dec = np.round(pmag.dotilt(correct_geo_dec, correct_geo_inc, bedding_strike-90, -bedding_dip), 1)[0]
        correct_strat_inc = np.round(pmag.dotilt(correct_geo_dec, correct_geo_inc, bedding_strike-90, -bedding_dip), 1)[1]
        
        correct_line = heading + '{:05.1f}'.format(correct_geo_dec) + ' '+ '{:05.1f}'.format(correct_geo_inc) \
        + ' ' + '{:05.1f}'.format(correct_strat_dec) + ' ' + '{:05.1f}'.format(correct_strat_inc) \
        + ' ' + moment_csd + '{:05.1f}'.format(this_spec_dec) + ' ' + '{:05.1f}'.format(this_spec_inc) + this_no_change_text
        corrected_file_lines.append(correct_line)
#     print(corrected_file_lines)
#     corrected_file = open('Jacobsville_corrected/'+this_file_path, 'w')
#     corrected_file.writelines(corrected_file_lines)
#     corrected_file.close()
    
        print(
#             correct_core_strike, correct_core_dip, 
#               bedding_strike, bedding_dip, 
              correct_geo_dec, correct_geo_inc,
              correct_strat_dec, correct_strat_inc, 
              this_spec_dec, this_spec_inc)

243.1 40.1 259.7 -11.5 254.7 37.5
243.4 38.1 258.1 -12.8 255.9 35.8
244.8 34.8 256.0 -15.6 258.7 33.1
239.8 33.8 252.8 -12.6 254.6 30.7
238.5 32.2 250.8 -12.5 254.1 28.8
232.3 33.2 249.3 -7.4 248.1 27.9
230.9 32.9 248.5 -6.5 247.0 27.3
235.4 31.6 249.0 -10.5 251.5 27.4
229.0 32.3 247.3 -5.3 245.5 26.2
154.3 -1.9 184.4 52.3 180.4 -19.9
219.9 2.5 216.2 -7.6 246.3 -4.8
229.2 30.6 245.8 -6.1 246.3 24.6
230.4 29.5 245.2 -7.5 247.8 23.9
228.5 29.7 244.7 -5.9 246.0 23.6
228.6 29.8 244.8 -6.0 246.1 23.7
227.8 30.0 244.7 -5.2 245.3 23.7
227.5 30.4 245.0 -4.8 244.9 24.0
226.8 30.8 245.2 -4.1 244.1 24.1
218.9 29.0 241.1 1.7 237.7 20.3
209.7 32.2 242.0 10.1 228.6 21.1
212.3 27.4 237.8 6.7 232.3 17.1
212.7 25.9 236.5 6.0 233.1 15.8
210.4 24.8 234.9 7.7 231.3 14.1
207.2 26.2 235.5 10.8 228.1 14.7
210.4 24.7 234.8 7.7 231.4 14.0
207.1 25.3 234.6 10.7 228.2 13.8
201.8 7.0 215.3 11.0 227.7 -5.1
205.3 14.7 223.7 9.7 229.2 3.1
210.1 33.8 243.6 10.2 228.5 22.7
207.0 22.3 231.6 10.1 228.9 10.9
206.2 23.2 2

In [4]:


this_file = open('Jacobsville_corrected/HF1-0')
lines = this_file.readlines()
lines

['HF1- 0 mag compass orientation (IGRF corrected)\n',
 '   80.6 338.8  78.0 208.5   9.0   1.0\n']